# <center>L2 Computational Physics</center>
---

## Week 5: Monte Carlo Methods

In this notebook, you will simulate a system with of three nuclei $A$, $B$ and $C$ where $A$ decays into $B$ and $B$ decays into $C$. If exposed to a neutron flux nucleus $C$ can be activated into a nucleus $A$. 

In [2]:
import numpy
from matplotlib import pyplot as plt
import random

Implement a function that tells whether a transition has occured, based on the transition probability and a random number. Use the random number `r` from `random.random()` and use the procedure described in the notes so that the checks can work in a reproducible way.

In [3]:
def has_transitioned(prob):
    r = random.random()
    if r < prob:
        return (True)
    else:
        return (False)
    # YOUR CODE HERE

In [4]:
# this test is worth 1 mark
random.seed(9867)
assert [ has_transitioned(0.5) for i in range(10)] == [False, False, True, False, False, False, False, True, False, True]

We are going to keep track of the state of the atoms using a either `'A'`, `'B'` or `'C'` for each nucleus. For example 
```python
state = ['A', 'A', 'B', 'A', 'C']
```

We will capture the possible physical reaction in a list of allowed transitions such as

`('A', 'B', 0.25 )`

where the first element is the initial state, the second element is the final state of the transition and its probability in the time interval considered is the third argument.

Define a function that takes as arguments the current state and a list of such transition rules and implements the transition (or not) and returns the new state

In [5]:
def evolveOne(currentState, rules):
    for i in range(0,len(rules)):
        (initial, final, probability) = rules[i]
        if currentState == initial:
            if has_transitioned(probability) == True:
                currentState = final
                return(currentState)
            else:
                currentState = initial
                return currentState
        else:
            currentState = currentState
    return currentState

In [6]:
# these tests are worth 1 mark
alwaysDecayRules = [
    ('A', 'B', 1.0),
    ('B', 'C', 1.0)
]
assert evolveOne('A', alwaysDecayRules) == 'B'
assert evolveOne('B', alwaysDecayRules) == 'C'

In [7]:
# these tests are worth 2 mark
random.seed(112211)
testRules = [
    ('A', 'B', 0.33),
    ('B', 'C', 0.75)
]
assert evolveOne('A', testRules) == 'A'
assert evolveOne('A', testRules) == 'A'
assert evolveOne('A', testRules) == 'A'
assert evolveOne('A', testRules) == 'A'
assert evolveOne('A', testRules) == 'B'

assert evolveOne('B', testRules) == 'B'
assert evolveOne('B', testRules) == 'C'
assert evolveOne('B', testRules) == 'C'
assert evolveOne('B', testRules) == 'C'
assert evolveOne('B', testRules) == 'C'

# with no rules there should be no change
assert evolveOne('C', testRules) == 'C'


Now implement a function that takes a list of states and transition them according to the rules passed as argument. This function should return a new vector of states, it should not modify the state passed as an argument!

In [8]:
def evolveMany(states, rules):
    newState = []
    for i in range(0, len(states)):
        Ans = evolveOne(states[i], rules)
        newState.append(Ans)
    return newState
    # YOUR CODE HERE

In [9]:
# these tests are worth 1 mark
random.seed(112287)
testRules = [
    ('A', 'B', 0.33),
    ('B', 'C', 0.75)
]
initialTestArray = ['A','B','C']*5
evolvedTest = evolveMany(initialTestArray, testRules)
targetArray = ['B', 'C', 'C', 'A', 'C', 'C', 'A', 'B', 'C', 'A', 'C', 'C', 'B', 'C', 'C'] 
assert evolvedTest == targetArray
# checks the initial array is left unchanged
assert initialTestArray == ['A','B','C']*5

Define a function that evolves a system that starts with initial amounts `NA`, `NB` and `NC` of $A$, $B$ and $C$ nuclei and evolved it in `n_timestep` from time $t=0$ to $t=t_{max}$. The function should return three arrays, one for each atom type, of the number of nuclei of that type at each time step. Each array should contain `n_timestep+1` elements including the initial amount. 
 

In [10]:
def evolve_system(NA, NB, NC, rules, n_step):
    A_count = [NA]
    B_count = [NB]
    C_count = [NC]
    state = (['A'] * NA)+(['B'] * NB)+(['C'] * NC)
    newstate = state
    for i in range(0,n_step):
        newstate = evolveMany(newstate, rules)
        NA = newstate.count('A')
        NB = newstate.count('B')
        NC = newstate.count('C')
        A_count.append(NA)
        B_count.append(NB)
        C_count.append(NC)
#(tmax-t0)/n = timestepquantity
#['A'] * NA = Initial amount of nuclei in state A
    #A_count = numpy.empty(n_step + 1, dtype=int)
    #B_count = numpy.empty(n_step + 1, dtype=int)
    #C_count = numpy.empty(n_step + 1, dtype=int)

    # YOUR CODE HERE
    return A_count, B_count, C_count

testrules = [
    ('A', 'B', 0.086),
    ('B', 'C', 0.075),
    ('C', 'A', 0.19)
    
]

random.seed(9485)
r1, r2, r3 = evolve_system(200,200,200, testrules, 20)
assert (r1 == [200, 213, 233, 250, 258, 251, 266, 263, 259, 260, 265, 259, 256,
        255, 258, 256, 259, 253, 249, 247, 253])

In [11]:
# these tests are worth 2 marks
rules = [
    ('A', 'B', 0.0033),
    ('B', 'C', 0.0075),
    ('C', 'A', 0.009)
    
]

r1, r2, r3 = evolve_system(0,0,250, rules, 17)
assert len(r1) == 18
assert len(r2) == 18
assert len(r3) == 18

In [12]:
# these tests are worth 2 marks 
testrules = [
    ('A', 'B', 0.086),
    ('B', 'C', 0.075),
    ('C', 'A', 0.19)
    
]

random.seed(9485)
r1, r2, r3 = evolve_system(200,200,200, testrules, 20)
assert (r1 == [200, 213, 233, 250, 258, 251, 266, 263, 259, 260, 265, 259, 256,
        255, 258, 256, 259, 253, 249, 247, 253]).all()
assert (r2 == [200, 198, 201, 206, 205, 214, 214, 212, 216, 221, 225, 234, 236,
        238, 234, 235, 231, 245, 253, 256, 252]).all()
assert (r3 == [200, 189, 166, 144, 137, 135, 120, 125, 125, 119, 110, 107, 108,
        107, 108, 109, 110, 102,  98,  97,  95]).all()

AttributeError: 'bool' object has no attribute 'all'

## Plotting tasks

Create a plot with the number of $A$, $B$ and $C$ nuclei, starting with 250 $C$ nuclei and evolving the system for 100 hours using 200 steps and with neutron flux on. Evolve the system for another 100 hours (using 200 steps) without neutron flux (i.e. no transitions from $C$ to $A$).  

The half life of the $A$ atoms is 10.1 hours, the half life of $B$ nuclei is 15.7 hours and we can caracterise the rate of activation of $C$ into $A$ when the neutron flux is on with and effective half-life of 3.2 hours.

The plot should have the appropriate labels and legend. [8 marks]    




In [ ]:
nsteps = 200
t_total = 100
t_half_A = 10.1
t_half_B = 15.7
t_half_C = 3.2
#p=delt/tau
#delt = time/steps

del_t = 100/200 #in hours
pA=(del_t)/10.1
pB=(del_t)/15.7
pC=(del_t)/3.2

rules1 = [
    ('A', 'B', pA),
    ('B', 'C', pB),
    ('C', 'A', pC)
]
rules2 = [
    ('A', 'B', pA),
    ('B', 'C', pB),
    ('C', 'A', 0)  
]
r1,r2,r3 = evolve_system(0, 0, 250, rules1, 199)
r4,r5,r6 = evolve_system(r1[199], r2[199], r3[199], rules2, 199)
plt.plot(numpy.arange(0,100,0.5),r1, color= 'blue', label = 'State A')
plt.plot(numpy.arange(0,100,0.5),r2, color= 'Red', label = 'State B')
plt.plot(numpy.arange(0,100,0.5),r3, color='Green', label = 'State C')
plt.plot(numpy.arange(100,200,0.5),r4, color= 'blue')
plt.plot(numpy.arange(100,200,0.5),r5, color= 'Red')
plt.plot(numpy.arange(100,200,0.5),r6, color='Green')
plt.legend()
plt.xlabel('Time (Hours)')
plt.ylabel('Number of nuclei in each state')
plt.title("Number of nuclei in each state of the system \n without neutron flux after the first 100 hours.")

Run the above simulation 20 times with 200 steps and use the results to calculate an average and the uncertainty on the number of $A$ atoms as a function of time. Use and `errorbar` plot for it. You might be interested in the `numpy.average` and `numpy.std` functions. The plot should have axis labels and a title.  [3 marks]


In [22]:
nsim = 20
del_t = 100/200 #in hours
pA=(del_t)/10.1
pB=(del_t)/15.7
pC=(del_t)/3.2

rules1 = [
    ('A', 'B', pA),
    ('B', 'C', pB),
    ('C', 'A', pC)
]
rules2 = [
    ('A', 'B', pA),
    ('B', 'C', pB),
    ('C', 'A', 0)  
]
r1,r2,r3 = evolve_system(0, 0, 250, rules1, 199)
r4,r5,r6 = evolve_system(r1[199], r2[199], r3[199], rules2, 199)
r1total = []
for i in range(0,20):
    r1total = []
    r1,r2,r3 = evolve_system(0, 0, 250, rules1, 199)
    r4,r5,r6 = evolve_system(r1[199], r2[199], r3[199], rules2, 199)
    r1[i]=numpy.array(r1)
    r1total.append(r1[i])
    print (r1total)
#how do i add elements of every list


[array([  0,  35,  56,  85,  94, 105, 121, 128, 142, 153, 154, 155, 158,
       158, 154, 154, 143, 138, 134, 135, 127, 125, 125, 123, 120, 122,
       122, 115, 108, 104, 100,  96,  99,  98,  93,  94,  97,  93,  93,
        92,  93,  95,  96,  97,  98,  99,  97,  96,  95,  88,  89,  90,
        90,  88,  89,  89,  86,  81,  84,  83,  83,  83,  83,  77,  78,
        87,  83,  83,  82,  86,  91,  94,  94,  95,  96,  94,  96,  96,
        94,  96,  92,  98,  96,  93,  91,  92,  90,  90,  90,  90,  88,
        88,  86,  80,  78,  80,  79,  78,  74,  73,  77,  80,  78,  79,
        79,  81,  88,  90,  88,  92,  92,  96,  98,  98,  93,  95,  96,
       102, 104, 102,  97,  94,  93,  92,  85,  85,  88,  88,  92,  88,
        85,  89,  87,  89,  90,  92,  93,  98,  98,  94,  96,  93,  90,
        88,  86,  85,  80,  82,  88,  91,  88,  88,  88,  86,  88,  85,
        85,  80,  79,  80,  78,  81,  79,  79,  80,  84,  85,  82,  83,
        84,  87,  81,  75,  75,  79,  78,  77,  75,  79,  79,  

[array([  0,  43,  70, 103, 123, 133, 142, 141, 142, 143, 150, 150, 153,
       151, 147, 149, 146, 144, 135, 128, 122, 121, 120, 118, 113, 107,
       104, 104, 108, 107, 108, 107, 109, 103, 102,  98,  92,  89,  92,
        91,  91,  90,  88,  87,  87,  85,  86,  87,  93,  93,  92,  87,
        91,  96,  97, 100, 100,  95,  98,  99,  97, 101,  96,  92,  92,
        92,  93,  93,  90,  89,  89,  89,  88,  88,  90,  93,  95,  99,
        99,  99,  97,  98, 100, 102,  98, 106, 105, 106, 109, 110, 110,
       109, 101,  97,  97,  92,  91,  88,  87,  85,  81,  82,  79,  82,
        86,  87,  86,  81,  83,  87,  88,  89,  87,  83,  80,  81,  78,
        84,  87,  87,  90,  91,  90,  93,  88,  90,  88,  89,  85,  88,
        89,  88,  85,  89,  93,  90,  92,  92,  93,  94,  95,  91,  86,
        84,  86,  83,  80,  84,  81,  74,  71,  77,  78,  77,  80,  79,
        80,  85,  86,  86,  86,  84,  85,  87,  90,  94,  98,  97,  95,
        92,  98,  92,  89,  90,  88,  87,  86,  86,  88,  88,  

[array([  0,  41,  66,  98, 119, 142, 143, 152, 155, 155, 155, 151, 147,
       146, 151, 149, 144, 141, 143, 139, 134, 133, 132, 128, 126, 124,
       121, 113, 111, 109, 107, 105, 100,  99,  91,  88,  88,  90,  86,
        83,  71,  75,  77,  81,  79,  78,  77,  79,  73,  78,  75,  77,
        74,  82,  83,  85,  87,  81,  85,  84,  81,  85,  89,  92,  89,
        90,  92,  91,  91,  85,  85,  82,  83,  87,  83,  82,  79,  82,
        82,  81,  82,  82,  84,  81,  81,  78,  83,  86,  89,  91,  93,
        98,  96,  93,  96,  93,  96,  98, 103, 101, 103, 105, 107, 104,
       104, 102, 104, 102,  95,  97,  98,  99, 102, 102, 105, 106, 110,
       109, 109, 109, 113, 105, 100,  94,  88,  92,  94,  94,  99, 101,
       108, 109, 110, 111, 106, 104, 101, 103, 105, 104, 102, 101,  97,
        94,  95,  91,  91,  93,  90,  89,  93,  83,  80,  77,  77,  78,
        78,  79,  79,  81,  82,  83,  80,  80,  77,  84,  84,  82,  81,
        83,  87,  86,  80,  84,  85,  87,  91,  87,  89,  86,  